# Model Creation and Training

In [ ]:
import pandas as pd
import numpy as np
import logging
import tensorflow as tf
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.layers import Lambda, Reshape, Concatenate, Add, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

logging.basicConfig(filename = "logs.log", format = "%(asctime)s -- %(message)s", datefmt='%m/%d/%Y %I:%M:%S %p', level = logging.INFO)

In [ ]:
### Load_Data ###
training_data = pd.read_csv("training_data/Attempt2.csv")
# training_data["Datetime"] = training_data["Datetime"].astype("datetime64").astype(int)
# training_data.set_index("Datetime", inplace=True)
training_data.head()
# percent_positive = (training_data["Response"].sum() / len(training_data)) * 100
X = training_data.iloc[:,:10]
y = training_data.iloc[:,10]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False)

logging.info("Dataset Upload successfully")
logging.info(f"X training set shape:{X_train.shape}")
logging.info(f"y training set shape:{y_train.shape}")
logging.info(f"X test set shape:{X_test.shape}")
logging.info(f"y test set shape:{y_test.shape}")
# logging.info(f"Percent Positive Response {round(percent_positive,2)}%")

print("Dataset Upload successfully")
print(f"X training set shape:{X_train.shape}")
print(f"y training set shape:{y_train.shape}")
print(f"X test set shape:{X_test.shape}")
print(f"y test set shape:{y_test.shape}")
# print(f"Percent Positive Response {round(percent_positive,2)}%")


# Batch Data

In [ ]:
### Global Variables ####
sequence_len = 10 # Looking at 24 hours worth of data to determine
epochs = 1
attention_heads = 4
projection_dim = sequence_len
dropout = 0.1
num_transformer_blocks = 2
mlp_units = [2048, 1024]
tranformer_mlp_units = [projection_dim ** 2, projection_dim * 2]


In [ ]:
#Not needed now
# batch_size = X_train.shape[0] // sequence_len
# max_num_samples = batch_size * sequence_len

# X_train = X_train.iloc[:max_num_samples, :]
X_train = tf.expand_dims(X_train, axis = 1)
# X_train = np.reshape(X_train, (batch_size, sequence_len, 2))

# y_train = y_train.iloc[:max_num_samples]
# y_train = tf.expand_dims(y_train, axis = -1)
# y_train = tf.expand_dims(y_train, axis = -1)
# y_train = np.reshape(y_train, (batch_size, -1))

In [ ]:
### Convert time to a vector that can be encoded to the features ###
class Time2Vector(tf.keras.layers.Layer):
    def __init__(self, kernal: int = 64):
        super().__init__(trainable = True,  name = "Time2VecLayer")
        self.kernal = kernal - 1
        
    def build(self, input_shape):
        ### Time to Vector Piecewise function ###
        
        self.weights_linear = self.add_weight(shape = (input_shape[1],1), initializer = "uniform", trainable = True, name = "weights_linear")
        
        self.bias_linear = self.add_weight(shape = (input_shape[1],1), initializer = "uniform", trainable = True, name = "bias_linear")
        
        self.weights_periodic = self.add_weight(shape = (input_shape[1], self.kernal), initializer = "uniform", trainable = True, name = "weights_periodic")
        
        self.bias_periodic = self.add_weight(shape = (input_shape[1], self.kernal), initializer = "uniform", trainable = True, name = "bias_periodic")
    
    def call(self, x):
        x = tf.expand_dims(x[:,:,0], axis = -1)
        time_linear = (self.weights_linear * x) + self.bias_linear
        # time_linear = tf.expand_dims(time_linear, axis = -1) #Expand dimensions to concat later
        
        time_periodic = tf.math.sin(tf.multiply(x, self.weights_periodic) + self.bias_periodic)
        # time_periodic = tf.expand_dims(time_periodic, axis = -1)
        
        final_product = tf.concat([time_linear, time_periodic], axis = -1)
        # final_product = tf.expand_dims(final_product, axis = 0)
        return final_product
    
    
test_vector = tf.random.uniform(shape = (5000, 1, 10), dtype = tf.float32)
exampleTV = Time2Vector(10)(test_vector)
exampleTV.shape
        

In [ ]:
### GPT-3.5 Verison of Time2Vec

class GPTTime2Vector(tf.keras.layers.Layer):
    def __init__(self, kernel: int = 64):
        super().__init__(trainable=True, name="Time2VecLayer")
        self.kernel = kernel - 1
        
    def build(self, input_shape):
        self.linear_weights = self.add_weight(shape=(input_shape[-1],),
                                               initializer=tf.keras.initializers.GlorotUniform(),
                                               trainable=True,
                                               name="linear_weights")
        
        self.linear_bias = self.add_weight(shape=(input_shape[-1],),
                                            initializer=tf.keras.initializers.GlorotUniform(),
                                            trainable=True,
                                            name="linear_bias")
        
        self.periodic_weights = self.add_weight(shape=(input_shape[-1],),
                                                 initializer=tf.keras.initializers.GlorotUniform(),
                                                 trainable=True,
                                                 name="periodic_weights")
        
        self.periodic_bias = self.add_weight(shape=(input_shape[-1],),
                                              initializer=tf.keras.initializers.GlorotUniform(),
                                              trainable=True,
                                              name="periodic_bias")
    
    def call(self, input_tensor):
        time_linear = (self.linear_weights * input_tensor[:, :, 0]) + self.linear_bias
        time_periodic = tf.math.sin(tf.multiply(input_tensor[:, :, 0], self.periodic_weights) + self.periodic_bias)
        output_tensor = tf.concat([time_linear, time_periodic], axis=-1)
        return output_tensor




batch_size = 32
sequence_length = 100
input_dim = 1

input_tensor = tf.random.normal(shape=(batch_size, sequence_length, input_dim))
GPTClass = GPTTime2Vector(kernel = 64)(input_tensor)
print(GPTClass.shape)


In [ ]:
hidden_size = 10
def mlp_block(x, units):
    x = GlobalAveragePooling1D(data_format = "channels_first")(x)
    for unit in units:
        x = Dense(unit, activation = tf.nn.gelu)(x)
        x = Dropout(0.1)(x)
    return x
def transformer_encoder(inputs, attention_heads, projection_dim, dropout):
    ### Layer Normalization / Multihead Attention Layers ###
    x = LayerNormalization(epsilon = 1e-6)(inputs)
    x = MultiHeadAttention(num_heads = attention_heads, key_dim = projection_dim, dropout = dropout)(x,x)
    skip1 = Add()([x, inputs])
    
    ### Feed Forward ###
    x = LayerNormalization(epsilon = 1e-6)(skip1)
    x = mlp_block(x, tranformer_mlp_units)
    skip2 = Add()([x,skip1])
    
    return skip2


def build_model():
    input = Input(shape = X_train.shape[1:]) # (Batch_size, Sequence Length, Number of Features)
    x = GPTTime2Vector(sequence_len)(input)
    x = Concatenate(axis = -1)([input, x]) 
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, attention_heads, projection_dim, dropout)
    x = mlp_block(x, mlp_units)
    output = Dense(1, activation = "relu")(x)
    
    model = Model(inputs = input, outputs = output)
    model.summary()

    return model








build_model()

In [ ]:
def train_model(model):
    optimizer = tf.optimizers.Adam(learning_rate=1e-3, decay = 1e-4)
    checkpoint_path = "/models/"
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = "val_Accuracy", save_best_only = True, save_weights_only = True)
    model.compile(optimizer=optimizer, 
                  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics = [tf.keras.metrics.CategoricalAccuracy(name = "Accuracy")])
    
    history = model.fit(
        x = X_train,
        y = y_train,
        epochs = epochs,
        batch_size = X_train.shape[0],
        callbacks = [checkpoint_callback],
        
    )
    
    return history

In [ ]:
model = build_model()
training = train_model(model)